<a href="https://colab.research.google.com/github/pietrzakkuba/computer-assisted-translation-labs/blob/main/lab_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Komputerowe wspomaganie tłumaczenia

# Zajęcia 2 - zaawansowane użycie pamięci tłumaczeń

Wiemy już, do czego służy pamięć tłumaczeń. Spróbujmy przeprowadzić mały research, którego celem będzie odkrycie, w jaki sposób do pamięci tłumaczeń podchodzą najwięksi producenci oprogramowania typu CAT.


### Ćwiczenie 1: Wykonaj analizę funkcjonalności pamięci tłumaczeń w programach SDL Trados Studio 2021 oraz Kilgray memoQ. Dla obu programów wypisz funkcje, które są związane z TM oraz zaznacz, które funkcje są wspólne dla obu programów oraz których funkcji Tradosa brakuje w memoQ oraz odwrotnie.

Odpowiedź:

```
memoQ:
- zachowuje informacje kontekstowe
- wspiera ponad 100 języków
- importowanie i eskportowanie bazy tłumaczeń
- concordance search
trados studio:
- wsparcie tłumaczenia siecią neuronową dostarczającą tłumaczenia w ponad 130 językach
- automatyczne sugestie zdań
- concordance search
- automatyczne poprawianie fuzzy matchów
```



Jedną z funkcji dostępnych we wszystkich większych programach do wspomagania tłumaczenia jest znajdowanie bardzo pewnych dopasowań w pamięci tłumaczeń. Są one zwane **ICE** (In-Context Exact match) lub 101% match. Są to takie dopasowania z pamięci tłumaczeń, dla których nie tylko zdanie źródłowe z TM jest identyczne z tłumaczonym, ale także poprzednie zdanie źródłowe z TM zgadza się z poprzednim zdaniem tłumaczonym oraz następne z TM z następnym tłumaczonym.

 Rozważmy przykładową pamięć tłumaczeń z poprzednich zajęć (można do niej dorzucić kilka przykładów):

In [ ]:
translation_memory = [
                      ('Wciśnij przycisk Enter', 'Press the ENTER button'), 
                      ('Sprawdź ustawienia sieciowe', 'Check the network settings'),
                      ('Drukarka jest wyłączona', 'The printer is switched off'),
                      ('Wymagane ponowne uruchomienie komputera', 'System restart required')
                     ]

### Ćwiczenie 2: Zaimplementuj funkcję ice_lookup, przyjmującą trzy parametry: aktualnie tłumaczone zdanie, poprzednio tłumaczone zdanie, następne zdanie do tłumaczenia. Funkcja powinna zwracać dopasowania typu ICE. Nie pozwól, aby doszło do błędów podczas sprawdzania pierwszego i ostatniego przykładu w pamięci (ze względu na brak odpowiednio poprzedzającego oraz następującego przykładu).

In [ ]:
def ice_lookup(sentence, prev_sentence=None, next_sentence=None):
    # sprawdzenie czy poprzednie i następne zdanie są podane (zakładam, że muszą)
    prev_exists = prev_sentence != '' and prev_sentence is not None
    next_exists = next_sentence != '' and next_sentence is not None
    translations = []
    # szukanie dopasowań
    for i in range(len(translation_memory)):
        # 1. sprawdzanie czy poprzednie się zgadza
        if not (prev_exists and i > 0 and translation_memory[i-1][0] == prev_sentence):
            continue
        # 2. sprawdzenie czy obecne się zgadza
        if not (translation_memory[i][0] == sentence):
            continue
        # 3. sprawdzenie czy następne sie zgadza
        if not (next_exists and i < len(translation_memory) - 1 and translation_memory[i+1][0] == next_sentence):
            continue
        translations.append(translation_memory[i][1])
    return translations
print(ice_lookup('Wciśnij przycisk Enter', None, 'Sprawdź ustawienia sieciowe'))
print(ice_lookup('Sprawdź ustawienia sieciowe', 'Wciśnij przycisk Enter', 'Drukarka jest wyłączona'))
print(ice_lookup('Drukarka jest wyłączona', 'Sprawdź ustawienia sieciowe', 'Wymagane ponowne uruchomienie komputera'))
print(ice_lookup('Wymagane ponowne uruchomienie komputera', 'Drukarka jest wyłączona', None))

[]
['Check the network settings']
['The printer is switched off']
[]


Inną powszechnie stosowaną techniką przeszukiwania pamięci tłumaczeń jest tzw. **fuzzy matching**. Technika ta polega na wyszukiwaniu zdań z pamięci, które są tylko podobne do zdania tłumaczonego. Na poprzednich zajęciach wykonywaliśmy funkcję tm_lookup, która pozwalała na różnicę jednego słowa.

Zazwyczaj jednak funkcje fuzzy match posiadają znacznie szersze możliwości. Ich działanie opiera się na zdefiniowaniu funkcji $d$ dystansu pomiędzy zdaniami $x$ i $y$. Matematycznie, funkcja dystansu posiada następujące właściwości:
1. $\forall_{x,y} d(x,y)\geqslant 0$
2. $\forall_{x,y} d(x,y)=0 \Leftrightarrow x=y$
3. $\forall_{x,y} d(x,y)=d(y,x)$
4. $\forall_{x,y,z} d(x,y) + d(y,z)\geqslant d(x,z)$

Rozważmy następującą funkcję:

In [ ]:
def sentence_distance(x,y):
    return abs(len(y) - len(x))

### Ćwiczenie 3: Czy to jest poprawna funkcja dystansu? Które warunki spełnia?

Odpowiedź: 

```
1 warunek: spełnia
2 warunek: nie spełnia - 2 różne słowa o tej samej długości
3 warunek: spełnia
4 warunek: spełnia, 
małe wyjaśnienie: żeby nie spełniała - d(x, z) > d(x, y) + d(y, z). Jest to sytuacja niemożliwa. 
W przypadku len(x/z) <= len(y) <= len(z/x) d(x, z) = d(x, y) + d(y, z).
W każdym innym przypadku 
(tzn. len(y) jest max lub min spośród długości x, y, z):
d(x, z) < d(x, y) + d(y, z)
```



A teraz spójrzmy na taką funkcję:

In [ ]:
def sentence_distance(x,y):
    if (x == y):
        return 0
    else:
        return 3

### Ćwiczenie 4: Czy to jest poprawna funkcja dystansu? Które warunki spełnia?

Odpowiedź:
```
1 warunek: spełnia
2 warunek: spełnia
3 warunek: spełnia
4 warunek: spełnia
```



Wprowadźmy jednak inną funkcję dystansu - dystans Levenshteina. Dystans Levenshteina pomiędzy dwoma łańcuchami znaków definiuje się jako minimalną liczbę operacji edycyjnych, które są potrzebne do przekształcenia jednego łańcucha znaków w drugi. Wyróżniamy trzy operacje edycyjne:
* dodanie znaku
* usunięcie znaku
* zamiana znaku na inny

### Ćwiczenie 5: Czy dystans Levenshteina jest poprawną funkcją dystansu? Uzasadnij krótko swoją odpowiedź sprawdzając każdy z warunków.

Odpowiedź:

W Pythonie dostępna jest biblioteka zawierająca implementację dystansu Levenshteina. Zainstaluj ją w swoim systemie przy użyciu polecenia:

`pip3 install python-Levenshtein`

I wypróbuj:

In [ ]:
# !pip install python-Levenshtein
from Levenshtein import distance as levenshtein_distance

levenshtein_distance("kotek", "kotki")


2

Funkcja ta daje nam możliwość zdefiniowania podobieństwa pomiędzy zdaniami:

In [ ]:
def levenshtein_similarity(x,y):
    return 1 - levenshtein_distance(x,y) / max(len(x), len(y))

Przetestujmy ją!

In [ ]:
levenshtein_similarity('Program jest uruchomiony', 'Program jest uruchamiany')

0.9166666666666666

In [ ]:
levenshtein_similarity('Spróbuj wyłączyć i włączyć komputer', 'Spróbuj włączyć i wyłączyć komputer')

0.9428571428571428

In [ ]:
levenshtein_similarity('Spróbuj wyłączyć i włączyć komputer', 'Nie próbuj wyłączać i włączać drukarki')

0.631578947368421

### Ćwiczenie 6: Napisz funkcję fuzzy_lookup, która wyszuka w pamięci tłumaczeń wszystkie zdania, których podobieństwo Levenshteina do zdania wyszukiwanego jest większe lub równe od ustalonego progu.

In [ ]:
def fuzzy_lookup(sentence, threshold):
    return [entry[1] for entry in translation_memory if levenshtein_similarity(entry[0], sentence) >= threshold]

fuzzy_lookup('Wciśnij przycisk Enterrr', 0.90)

['Press the ENTER button']